In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config('spark.driver.maxResultSize', '18g').getOrCreate()

In [0]:
import json
import pandas as pd
import numpy as np
# Import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F

# Import SparkNLP
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *

In [0]:
year = 2014
month_literal = str(2)
data_path = "/mnt/lsde/group05/kwfilter_top3/" + str(year) + "-" + month_literal + ".parquet"
df =spark.read.load(data_path,format="parquet")
df.show()

+------------------+--------------------+--------------------+-----------------+------+-----+-----------+
 author| title| selftext| subreddit| id|score|created_utc|
+------------------+--------------------+--------------------+-----------------+------+-----+-----------+
 vordhosbn_|Russian bear come...| | videos|1x4dik| 1| 1391637494|
 benjaminsanders|Balloon Invasion!...| | technology|1x4ebb| 3| 1391637925|
 LawrenceLongshot|NATO raps Russia ...| | worldnews|1x4ehc| 4| 1391638006|
 LatinArma|Russia is clearly...|So I've noticed w...| circlebroke|1x4fa5| 4| 1391638438|
 goobleber|It had been three...| | MakeStories|1x4fje| 3| 1391638572|
 relevantlife|The United States...| | worldnews|1x4fxb| 1| 1391638771|
 BabyMonkeyJR|Question about th...|Is there any spec...| Warthunder|1x4geo| 1| 1391639022|
 vilnius2013|Feds warn of toot...| | news|1x4giv| 1| 1391639085|
 Jerjacques|Russia becomes 3r...| | worldnews|1x4h1e| 1| 1391639362|
 PM_ME_YOUR_PASTA|Will you be watch...|Does all that has...| AskReddit|1x4hpa| 1| 1391639706|
 Hilby|DAE think that it...| | DoesAnybodyElse|1x4hv2| 1| 1391639785|
 gigiandvin|Homeland Security...| | worldnews|1x4hzl| 2| 1391639853|
 sigurbodi91|widespread violen...| | videos|1x4i3y| 1| 1391639925|
 heybabyzr0|Russian Woman in ...| | pantyhose|1x4i4f| 1| 1391639930|
 acore8604|After hearing tha...| | AdviceAnimals|1x4i6i| 18| 1391639965|
 KinkyJFK|Russia Anti gay l...| | AskReddit|1x4j8t| 1| 1391640562|
 Jiveturtle|Vladimir Putin st...| | funny|1x4jpn| 0| 1391640828|
 BladeSplitter12| How's Ukraine doin?|There was a week ...| AskReddit|1x4jvv| 3| 1391640926|
Whoa_This_is_heavy|After watching 'H...| | AdviceAnimals|1x4kpn| 2| 1391641371|
 Blomune|(short film) Зако...| |fullforeignmovies|1x4mbc| 2| 1391642263|
+------------------+--------------------+--------------------+-----------------+------+-----+-----------+
only showing top 20 rows

In [0]:
schema_c =  df.schema

In [0]:
schema_c

Out[11]: StructType(List(StructField(parent_id,StringType,true),StructField(title,StringType,true),StructField(selftext,StringType,true),StructField(author,StringType,true),StructField(body,StringType,true),StructField(id,StringType,true),StructField(score,LongType,true),StructField(ups,LongType,true),StructField(subreddit,StringType,true),StructField(subreddit_id,StringType,true),StructField(created_utc,StringType,true)))

In [0]:
def handle(year_range = [2015, 2016, 2017], month_range = range(1, 13)):
    for year in year_range:
        df_samples = spark.createDataFrame([], schema_c)
        for month in month_range:
            n=300
            month_literal = str(month)
            data_path = "/mnt/lsde/group05/kwfilter_top3/" + str(year) + "-" + month_literal + ".parquet"
            print("start handling "+str(year) + "-" + month_literal)
            df =spark.read.load(data_path,format="parquet")
            df_sam = df.sample(fraction=0.01,seed = 123).limit(n)
            df_samples = df_samples.union(df_sam)
            print("finish handling "+str(year) + "-" + month_literal)
        df_samples.write.mode("overwrite").format("parquet").save("/mnt/lsde/group05/"+"submission_sample/"+str(year)+".parquet")
            

In [0]:
data_path = "/mnt/lsde/group05/comments_sample_2/" + str(2014)  + ".parquet"
df1 =spark.read.load(data_path,format="parquet")
df1.show()

+---------+--------------------+--------------------+--------------------+--------------------+-------+-----+---+------------------+------------+-----------+
parent_id| title| selftext| author| body| id|score|ups| subreddit|subreddit_id|created_utc|
+---------+--------------------+--------------------+--------------------+--------------------+-------+-----+---+------------------+------------+-----------+
t3_1za4vu|Russian soldiers ...| | Castigar2000|Putin should fix ...|cfs8qft| -1| -1| pics| t5_2qh0u| 1393737437|
t3_1zbkia|Canadian Prime Mi...| | Figur3z|Anyone know why t...|cfsh85x| 1| 1| worldnews| t5_2qh13| 1393777021|
t3_1zchji|Kerry: Russia beh...| | Kindle282|I've never seen a...|cfshh49| 2| 2| worldnews| t5_2qh13| 1393777756|
t3_1zcp9y|Just putin it out...| | smackthem|ITT http://i.imgu...|cfsis9v| 5| 5| funny| t5_2qh33| 1393781278|
t3_20xsis|Is Putin eyeing E...| | AtomicSteve21|I'm sure Putin wa...|cg810yf| 3| 3| geopolitics| t5_2qhnn| 1395375135|
t3_1z8wpl|So it looks like ...| | LordoftheGodKings|I believe with al...|cfro1ml| 23| 23| AdviceAnimals| t5_2s7tt| 1393674942|
t3_1z9j57|Putin ask for Aut...| | tauntology|If Putin limits h...|cfrvl0w| 1| 1| worldnews| t5_2qh13| 1393701708|
t3_1z9j57|Putin ask for Aut...| | lo0ilo0ilo0i|i bet russia's vo...|cfrvmpt| 1| 1| worldnews| t5_2qh13| 1393701827|
t3_1z9j57|Putin ask for Aut...| | Ozercc| God damnit Russia|cfrwqml| 1| 1| worldnews| t5_2qh13| 1393704718|
t3_1zad80|Ukraine military ...| |niggaitwastimefoanu1|We are now a sold...|cfrxcbz| -10|-10| worldnews| t5_2qh13| 1393706298|
t3_1ztn66|Another reason wh...| | iprotectandserve|We don't have the...|cfx9bc6| 0| 0| Bad_Cop_No_Donut| t5_2qqdb| 1394249592|
t3_1zutdg|Animals left to s...| | naoptovke|This is why zoos ...|cfxi9r9| 1| 1| worldnews| t5_2qh13| 1394291019|
t3_219fnt|From Russia with ...| | Kjartanski|The lion of the P...|cgeh77k| 1| 1| TankPorn| t5_2skj2| 1396043545|
t3_1zquyg|ELI5: Why is Russ...| | AutoModerator|This question is ...|cfw330l| 1| 1| explainlikeimfive| t5_2sokd| 1394136064|
t3_1zon1v|My Russian wife w...| | 47attemptslater|I am a photograph...|cfw7j9t| 3| 3| funny| t5_2qh33| 1394145087|
t3_1zp2w0|A one month old a...|[Here's](http://w...| texdeveloper|Listen to me, the...|cfwgrwl| 1| 1| conspiracy| t5_2qh4r| 1394165894|
t3_207hfn|Ukraine aid bill ...| | VerVeritas|1) Although I'm p...|cg0j1a0| 0| 0| UkrainianConflict| t5_30hhs| 1394605887|
t3_207wwd|Hey guys, I'm you...| | damisword|Thanks for the op...|cg0lnbn| 3| 3|Anarcho_Capitalism| t5_2qorf| 1394620830|
t3_21bvud|Putin doesn't tak...| | Falconfan018|I mean, I feel st...|cgblue6| 1| 1| funny| t5_2qh33| 1395769087|
t3_21ohtp|TIL that a Russia...| | Itsnotironic444|His eyes remind m...|cgfgcr3| 1| 1| todayilearned| t5_2qqjc| 1396154999|
+---------+--------------------+--------------------+--------------------+--------------------+-------+-----+---+------------------+------------+-----------+
only showing top 20 rows

In [0]:
handle([2014],[2,3])

start handling 2014-2
finish handling 2014-2
start handling 2014-3
finish handling 2014-3

In [0]:
def handle2022(year_range = [2015, 2016, 2017], month_range = range(1, 13)):
    for year in year_range:
        #df_samples = spark.createDataFrame([], schema_c)
        for month in month_range:
            n=500
            month_literal = str(month)
            data_path = "/mnt/lsde/group05/kwfilter_top3/" + str(year) + "-" + month_literal + ".parquet"
            print("start handling "+str(year) + "-" + month_literal)
            df =spark.read.load(data_path,format="parquet")
            df_sam = df.sample(fraction=0.01,seed = 123).limit(n)
            #df_samples = df_samples.union(df_sam)
            df_sam.write.mode("overwrite").format("parquet").save("/mnt/lsde/group05/"+"submission_sample/"+str(year)+ "-" + month_literal+".parquet")
            print("finish handling "+str(year) + "-" + month_literal)

In [0]:
handle2022([2022],[2,3,4])

start handling 2022-2
finish handling 2022-2
start handling 2022-3
finish handling 2022-3
start handling 2022-4
finish handling 2022-4